# Change of Chararacter CHOCH

### Load the data

In [23]:
import pandas as pd
import pandas_ta as ta
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import stats

df = pd.read_csv("downloads/EURUSD.csv")
df.columns = map(str.lower, df.columns)
df=df[df['open']!=0]
df.reset_index(drop=True, inplace=True)

df['RSI'] = ta.rsi(df.close, length=12)
df['EMA'] = ta.ema(df.close, length=150)
df.tail()

df=df

### Trend detection

In [24]:
EMAsignal = [0]*len(df)
backcandles = 15

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df.open[i], df.close[i])>=df.EMA[i]:
            dnt=0
        if min(df.open[i], df.close[i])<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        EMAsignal[row]=3
    elif upt==1:
        EMAsignal[row]=2
    elif dnt==1:
        EMAsignal[row]=1

df['EMASignal'] = EMAsignal

In [25]:
def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, 2 if pivot low, 3 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0
    
    pivotHigh = 1
    pivotLow = 2
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle].low > df.iloc[i].low:
            pivotLow=0
        if df.iloc[candle].high < df.iloc[i].high:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

In [26]:
window=5
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window), axis=1)

In [27]:
def pointpos(x):
    if x['isPivot']==2:
        return x['low']-1e-3
    elif x['isPivot']==1:
        return x['high']+1e-3
    else:
        return np.nan
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [28]:
dfpl = df
fig = go.Figure(data=[go.Candlestick(x=dfpl.date,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.date, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [29]:
def detect_structure(candle, backcandles, window):
    """
    Attention! window should always be greater than the pivot window! to avoid look ahead bias
    """
    localdf = df[candle-backcandles-window:candle-window]  
    highs = localdf[localdf['isPivot'] == 1].high.tail(3).values
    idxhighs = localdf[localdf['isPivot'] == 1].high.tail(3).index
    lows = localdf[localdf['isPivot'] == 2].low.tail(3).values
    idxlows = localdf[localdf['isPivot'] == 2].low.tail(3).index

    pattern_detected = False

    lim1 = 0.005
    lim2 = lim1/3
    if len(highs) == 3 and len(lows) == 3:
        order_condition = (idxlows[0] < idxhighs[0] 
                           < idxlows[1] < idxhighs[1] 
                           < idxlows[2] < idxhighs[2])
        diff_condition = ( 
                            abs(lows[0]-highs[0])>lim1 and 
                            abs(highs[0]-lows[1])>lim2 and
                            abs(highs[1]-lows[1])>lim1 and
                            abs(highs[1]-lows[2])>lim2
                            )
        pattern_1 = (lows[0] < highs[0] and
            lows[1] > lows[0] and lows[1] < highs[0] and
            highs[1] > highs[0] and
            lows[2] > lows[1] and lows[2] < highs[1] and
            highs[2] < highs[1] and highs[2] > lows[2]
            )

        pattern_2 = (lows[0] < highs[0] and
            lows[1] > lows[0] and lows[1] < highs[0] and
            highs[1] > highs[0] and
            lows[2] < lows[1] and
            highs[2] < highs[1] 
            )

        if (order_condition and
            diff_condition and
            (pattern_1 or pattern_2)
        ):
            pattern_detected = True

    if pattern_detected:
        return 1
    else:
        return 0

In [30]:
df['pattern_detected'] = df.index.map(lambda x: detect_structure(x, backcandles=40, window=6))

In [31]:
df[df['pattern_detected']!=0]

,date,open,high,low,close,adj close,volume,bbl,bbm,bbh,...,bbp_30_2.0,rsi,atr,bb_width,RSI,EMA,EMASignal,isPivot,pointpos,pattern_detected
3691,2023-03-14 19:00:00,1.0746,1.0746,1.0738,1.0743,1.0743,0.0,1.0694,1.0723,1.0752,...,0.8545,59.6540,0.0019,0.0054,60.137771,1.065428,2,0,NaN,1
8410,2023-12-15 11:00:00,1.0966,1.0971,1.0961,1.0971,1.0971,0.0,1.0899,1.0970,1.1041,...,0.5071,52.7331,0.0015,0.0130,50.442360,1.086457,2,0,NaN,1
8411,2023-12-15 12:00:00,1.0971,1.0972,1.0945,1.0945,1.0945,0.0,1.0903,1.0971,1.1039,...,0.3029,42.8696,0.0016,0.0124,39.603643,1.086563,2,0,NaN,1
8412,2023-12-15 13:00:00,1.0945,1.0952,1.0909,1.0921,1.0921,0.0,1.0910,1.0972,1.1035,...,0.0872,36.2599,0.0018,0.0114,32.558721,1.086637,2,0,NaN,1
8413,2023-12-15 14:00:00,1.0922,1.0930,1.0916,1.0925,1.0925,0.0,1.0913,1.0973,1.1033,...,0.1023,38.3053,0.0017,0.0109,34.671627,1.086714,2,0,NaN,1
8414,2023-12-15 15:00:00,1.0924,1.0924,1.0909,1.0917,1.0917,0.0,1.0915,1.0973,1.1032,...,0.0197,36.1217,0.0017,0.0107,32.453270,1.086780,2,0,NaN,1
8415,2023-12-15 16:00:00,1.0918,1.0918,1.0905,1.0908,1.0908,0.0,1.0913,1.0973,1.1033,...,-0.0482,33.7573,0.0017,0.0109,30.090468,1.086834,2,0,NaN,1
